In [3]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
spot_df = pd.read_csv("./ArchivosRendimientos/Benchmarks/Historico_SPOT.csv")
spot_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"Spot"}, inplace=True)
spot_df = spot_df[["Fecha", "Spot"]]
spot_df["Fecha"] = pd.to_datetime(spot_df["Fecha"])
spot_df

,Fecha,Spot
0,2024-01-02,17.0440
1,2024-01-03,17.0364
2,2024-01-04,17.0348
3,2024-01-05,16.8832
4,2024-01-08,16.8100
...,...,...
300,2025-03-07,20.2672
301,2025-03-10,20.3623
302,2025-03-11,20.2582
303,2025-03-12,20.1797


In [3]:
precios_bmks_isimp_df = pd.read_excel("./ArchivosPeergroups/Benchmarks/Historico_ISIMP.xlsx", skiprows=2, skipfooter=4)
precios_bmks_isimp_df["Fecha"] = pd.to_datetime(precios_bmks_isimp_df["Fecha"], format="%d/%m/%Y")
precios_bmks_isimp_df.rename(columns={"Índice":"ISIMP"}, inplace=True)
precios_bmks_isimp_df = precios_bmks_isimp_df[["Fecha", "ISIMP"]]
precios_bmks_isimp_df

,Fecha,ISIMP
0,2024-01-02,260.585246
1,2024-01-03,260.588146
2,2024-01-04,260.656244
3,2024-01-05,260.673049
4,2024-01-08,260.905944
...,...,...
296,2025-03-07,291.468154
297,2025-03-10,291.634970
298,2025-03-11,291.702763
299,2025-03-12,291.773025


In [4]:
precios_bmks_acwi_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Historico_ACWI.csv")
precios_bmks_acwi_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"ACWI"}, inplace=True)
precios_bmks_acwi_df["Fecha"] = pd.to_datetime(precios_bmks_acwi_df["Fecha"], format="%Y-%m-%d")
precios_bmks_acwi_df = precios_bmks_acwi_df[["Fecha", "ACWI"]]
precios_bmks_acwi_df

,Fecha,ACWI
0,2024-01-02,1719.739600
1,2024-01-03,1705.343640
2,2024-01-04,1703.309652
3,2024-01-05,1689.670656
4,2024-01-08,1702.516800
...,...,...
300,2025-03-07,2421.930400
301,2025-03-10,2370.375343
302,2025-03-11,2345.899560
303,2025-03-12,2350.733253


In [8]:
precios_bmks_valmer_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Benchmarks_SP_Historico_MD.csv")
precios_bmks_valmer_df.rename(columns={"FECHA":"Fecha"}, inplace=True)
precios_bmks_valmer_df["Fecha"] = pd.to_datetime(precios_bmks_valmer_df["Fecha"], format="%Y%m%d")
precios_bmks_valmer_df.drop(columns=[col for col in precios_bmks_valmer_df.columns if "Unnamed" in col], inplace=True)
precios_bmks_valmer_df.set_index("Fecha", inplace=True)
precios_bmks_valmer_df

,S&P/BMV Government MBONOS 5-10 Year Bond Index,S&P/BMV Government MBONOS 1-5 Year Bond Index,S&P/BMV Government CETES Bond Index,S&P/BMV Government Inflation-Linked UDIBONOS 1+ Year Bond Index,S&P/BMV Mexico Sovereign Bond Index,S&P/BMV Mexico Sovereign Inflation-Linked Bond Index,S&P/BMV Sovereign MBONOS Bond Index,S&P/BMV Sovereign MBONOS 1-3 Year Bond Index,S&P/BMV Sovereign MBONOS 3-5 Year Bond Index,S&P/BMV Sovereign MBONOS 10-20 Year Bond Index,...,S&P/BMV Sovereign UDIBONOS & CBICS Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 1-3 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 3-5 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 5-10 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 10+ Year Bond Index,S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (MXN),S&P/BMV Sovereign BONDESF Bond Index (MXN),S&P/BMV Sovereign BONDES Bond Index (MXN),S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (USD),S&P/BMV Green Social & Sustainable Target Duration Bond Index (MXN)
Fecha,,,,,,,,,,,,,,,,,,,,,
2001-01-04,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN,NaN,NaN,NaN,100.092145,100.090847,100.093364,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-08,NaN,NaN,NaN,NaN,NaN,NaN,100.277668,100.325108,100.233323,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-09,NaN,NaN,NaN,NaN,NaN,NaN,100.390324,100.480925,100.305671,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-01-10,NaN,NaN,NaN,NaN,NaN,NaN,100.310827,100.491887,100.141794,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,494.171129,436.812220,398.230100,510.326162,449.001594,493.051046,771.591870,656.562229,797.044142,328.336458,...,137.931759,154.012112,144.564503,142.838602,127.179754,133.978840,140.023744,140.009830,122.686301,159.619967
2025-03-10,493.545160,437.081818,398.521394,510.870043,448.788226,493.572850,770.857436,657.172515,797.175835,327.382786,...,138.080240,154.237078,144.822728,143.120338,127.242343,134.138825,140.135766,140.121913,122.777112,159.641775
2025-03-11,492.509318,437.250243,398.672409,510.958887,448.847727,493.667112,770.922666,657.557922,797.250822,327.665437,...,138.110771,154.479999,144.809636,143.331202,127.176562,134.191050,140.175076,140.161201,122.543498,159.561361


In [ ]:
propiedades_fondos_path = "./ArchivosRendimientos/PropiedadesFondos"
propiedades_fondos_filename = os.listdir(propiedades_fondos_path)[0]
propiedades_fondos_df = pd.read_excel(os.path.join(propiedades_fondos_path, propiedades_fondos_filename))
fondo2tipo_fondo = {fondo:propiedades_fondos_df[propiedades_fondos_df["Fondo"] == fondo]["Tipo"].item() for fondo in propiedades_fondos_df["Fondo"]}

{'VECTRF': 'RF',
 'VECTPRE': 'RF',
 'VECTMD': 'RF',
 'VECTFI': 'RF',
 'VECTCOR': 'RF',
 'VECTTR': 'RF',
 'VECTPZO': 'RF',
 'VECTCOB': 'RV',
 'INCOME': 'RF',
 'VECTIND': 'RV',
 'VECTPA': 'RV',
 'VECTMIX': 'RV',
 'VECTSIC': 'RV',
 'VECTUSD': 'RV',
 'COMMODQ': 'RV',
 'VECTUSA': 'RV',
 'MXRATES': 'RV',
 'EQUITY': 'RV',
 'NEXTGEN': 'RV',
 'BALANCE': 'RV',
 'DYNAMIC': 'RV'}

In [ ]:
hoy = datetime.today()
hoy = datetime(year=hoy.year, month=hoy.month, day=hoy.day)
hoy

datetime.datetime(2025, 3, 20, 0, 0)

In [ ]:
hoy - timedelta(days=180)

datetime.datetime(2024, 9, 21, 0, 0)

In [15]:
(hoy - timedelta(days=1)) - timedelta(days=180)

datetime.datetime(2024, 9, 20, 0, 0)

In [17]:
datetime(year=2025, month=3, day=18) - timedelta(days=180)

datetime.datetime(2024, 9, 19, 0, 0)

In [19]:
datetime(year=2025, month=3, day=14) - timedelta(days=180)

datetime.datetime(2024, 9, 15, 0, 0)

In [25]:
9881.407764160156/10793.744421032714 - 1

-0.0845245747244836